<a href="https://colab.research.google.com/github/MohammadErfanRashidi/Tweet-Classification/blob/main/Tweet%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [343]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [344]:
# Downloading the stopwords
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [345]:
# Uploading the train data
dataset = pd.read_csv("/content/train (1).csv")

In [346]:
# Uploading the test data
dataset_test = pd.read_csv("/content/test (1).csv")

In [347]:
# Checking the head
dataset.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [348]:
# Checking the head test
dataset_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [349]:
# Checking the tail
dataset.tail()

,id,keyword,location,text,target
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,1


In [350]:
# Checking the head test
dataset_test.tail()

,id,keyword,location,text
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...
3262,10875,NaN,NaN,#CityofCalgary has activated its Municipal Eme...


In [351]:
# Checking the shape
dataset.shape

(7613, 5)

In [352]:
# Checking the shape test
dataset_test.shape

(3263, 4)

In [353]:
# Getting the info
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [354]:
# Getting the info test
dataset_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [355]:
# Checking for missing values
dataset.isnull().sum()

,0
id,0
keyword,61
location,2533
text,0
target,0


In [356]:
# Checking for missing values test
dataset_test.isnull().sum()

,0
id,0
keyword,26
location,1105
text,0


In [357]:
# Getting the value counts of target
dataset['target'].value_counts()

,count
target,
0,4342
1,3271


1: Real disaster

0: Fake disaster

In [358]:
# Dropping the missing values
dataset = dataset.dropna(subset=['keyword'])
dataset = dataset.dropna(subset=['location'])

In [359]:
# Dropping the missing values test
dataset_test = dataset_test.dropna(subset=['keyword'])
dataset_test = dataset_test.dropna(subset=['location'])

In [360]:
# Checking the missing values again
dataset.isnull().sum()

,0
id,0
keyword,0
location,0
text,0
target,0


In [361]:
# Checking the missing values test
dataset_test.isnull().sum()

,0
id,0
keyword,0
location,0
text,0


In [362]:
# Getting the info
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5080 entries, 31 to 7581
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5080 non-null   int64 
 1   keyword   5080 non-null   object
 2   location  5080 non-null   object
 3   text      5080 non-null   object
 4   target    5080 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 238.1+ KB


In [363]:
# Merging keyword, location, and text columns together
dataset['content'] = dataset['keyword'] + ' ' + dataset['location'] + ' ' + dataset['text']

In [364]:
# Merging keyword, location, and text columns together test
dataset_test['content'] = dataset_test['keyword'] + ' ' + dataset_test['location'] + ' ' + dataset_test['text']

In [365]:
# Checking the new column content
dataset['content']

,content
31,ablaze Birmingham @bbcmtd Wholesale Markets ab...
32,ablaze Est. September 2012 - Bristol We always...
33,ablaze AFRICA #AFRICANBAZE: Breaking news:Nige...
34,"ablaze Philadelphia, PA Crying out for more! S..."
35,"ablaze London, UK On plus side LOOK AT THE SKY..."
...,...
7575,wrecked TN On the bright side I wrecked http:/...
7577,wrecked #NewcastleuponTyne #UK @widda16 ... He...
7579,"wrecked Vancouver, Canada Three days off from ..."
7580,wrecked London #FX #forex #trading Cramer: Ig...


In [366]:
# Checking the new column content
dataset_test['content']

,content
15,ablaze London Birmingham Wholesale Market is a...
16,ablaze Niall's place | SAF 12 SQUAD | @sunkxss...
17,ablaze NIGERIA #PreviouslyOnDoyinTv: Toke Maki...
18,ablaze Live On Webcam Check these out: http://...
19,"ablaze Los Angeles, Califnordia PSA: IÛªm spl..."
...,...
3246,wrecked Love Reiss @yakubOObs think he deactiv...
3247,wrecked Seattle Washington RT CNBC '3 words fr...
3248,wrecked Acey mountain islanddåÇTorontoåÈ Smack...
3249,wrecked los angeles @thrillhho jsyk I haven't ...


In [367]:
# Stemming words
port_stem = PorterStemmer()

In [368]:
# Creating the Stemming function
def stemming(content):
    stemmed_content = re.sub("[^a-zA-Z]", " ", content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words("english")]
    stemmed_content = " ".join(stemmed_content)
    return stemmed_content

In [369]:
# Stemming the "content" column in our dataset
dataset["content"] = dataset["content"].apply(stemming)

In [370]:
# Stemming the "content" column in our dataset test
dataset_test["content"] = dataset_test["content"].apply(stemming)

In [371]:
# Printing the new content column
dataset["content"]

,content
31,ablaz birmingham bbcmtd wholesal market ablaz ...
32,ablaz est septemb bristol alway tri bring heav...
33,ablaz africa africanbaz break news nigeria fla...
34,ablaz philadelphia pa cri set ablaz
35,ablaz london uk plu side look sky last night a...
...,...
7575,wreck tn bright side wreck http co uea txrhi
7577,wreck newcastleupontyn uk widda gone relax tho...
7579,wreck vancouv canada three day work pretti muc...
7580,wreck london fx forex trade cramer iger word w...


In [372]:
# Printing the new content column test
dataset_test["content"]

,content
15,ablaz london birmingham wholesal market ablaz ...
16,ablaz niall place saf squad sunkxssedharri wea...
17,ablaz nigeria previouslyondoyintv toke makinwa...
18,ablaz live webcam check http co roi nsmejj htt...
19,ablaz lo angel califnordia psa split person te...
...,...
3246,wreck love reiss yakuboob think deactiv notif ...
3247,wreck seattl washington rt cnbc word disney ce...
3248,wreck acey mountain islandd toronto smackdown ...
3249,wreck lo angel thrillhho jsyk stop think abt r...


In [373]:
# Separating the data and label
X_train = dataset["content"]
Y_train = dataset["target"]

In [374]:
# Separating the data test
X_test = dataset_test["content"]

In [375]:
print(X_train)

31      ablaz birmingham bbcmtd wholesal market ablaz ...
32      ablaz est septemb bristol alway tri bring heav...
33      ablaz africa africanbaz break news nigeria fla...
34                    ablaz philadelphia pa cri set ablaz
35      ablaz london uk plu side look sky last night a...
                              ...                        
7575         wreck tn bright side wreck http co uea txrhi
7577    wreck newcastleupontyn uk widda gone relax tho...
7579    wreck vancouv canada three day work pretti muc...
7580    wreck london fx forex trade cramer iger word w...
7581    wreck lincoln enginesh great atmospher british...
Name: content, Length: 5080, dtype: object


In [376]:
print(X_test)

15      ablaz london birmingham wholesal market ablaz ...
16      ablaz niall place saf squad sunkxssedharri wea...
17      ablaz nigeria previouslyondoyintv toke makinwa...
18      ablaz live webcam check http co roi nsmejj htt...
19      ablaz lo angel califnordia psa split person te...
                              ...                        
3246    wreck love reiss yakuboob think deactiv notif ...
3247    wreck seattl washington rt cnbc word disney ce...
3248    wreck acey mountain islandd toronto smackdown ...
3249    wreck lo angel thrillhho jsyk stop think abt r...
3250    wreck brussel belgium stighefootbal begov garb...
Name: content, Length: 2158, dtype: object


In [377]:
print(Y_train)

31      1
32      0
33      1
34      0
35      0
       ..
7575    0
7577    0
7579    0
7580    0
7581    0
Name: target, Length: 5080, dtype: int64


In [378]:
# Numerating the text
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)  # Fit only on training data

X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)  # Transform testing data using the same fitted vectorizer

In [379]:
print(X_train)

  (0, 36)	0.5735303667983405
  (0, 1011)	0.4134067612700405
  (0, 1274)	0.3133268751649312
  (0, 2373)	0.07584976037163814
  (0, 6084)	0.07602150076320423
  (0, 7780)	0.4134067612700405
  (0, 8305)	0.28676518339917023
  (0, 15046)	0.3705565570804108
  (1, 36)	0.24562160833782168
  (1, 362)	0.24562160833782168
  (1, 1576)	0.26391719315817735
  (1, 1579)	0.29384707379743813
  (1, 2373)	0.0649672317736953
  (1, 4098)	0.35409331215614737
  (1, 5749)	0.26608287074034415
  (1, 6084)	0.06511433174829127
  (1, 8518)	0.28962682904572334
  (1, 11775)	0.21111468743753461
  (1, 12178)	0.29856490604195945
  (1, 13905)	0.22436714619253906
  (1, 15449)	0.35409331215614737
  (1, 15566)	0.35409331215614737
  (2, 14)	0.2965889293159246
  (2, 36)	0.521992230189384
  (2, 162)	0.25443507536677984
  :	:
  (5078, 2684)	0.30013976440004064
  (5078, 3315)	0.27027446668507715
  (5078, 3985)	0.33484715236613255
  (5078, 4666)	0.33484715236613255
  (5078, 4898)	0.29323376307741816
  (5078, 6084)	0.061575149305627

In [380]:
print(X_test)

  (0, 36)	0.41210743727716664
  (0, 1008)	0.2504682285945762
  (0, 1274)	0.45027898444193115
  (0, 1542)	0.19169046343071178
  (0, 2373)	0.05450147398356281
  (0, 4515)	0.13834762076544546
  (0, 6084)	0.05462487720114721
  (0, 7960)	0.15864080537884773
  (0, 8305)	0.41210743727716664
  (0, 9245)	0.15576255473607592
  (0, 15046)	0.5325231999733064
  (1, 36)	0.5176384666209414
  (1, 9283)	0.35600561541714587
  (1, 10443)	0.263266238272343
  (1, 11140)	0.33444493806704256
  (1, 11881)	0.35600561541714587
  (1, 12324)	0.32024328295511945
  (1, 12804)	0.30963592684474445
  (1, 14916)	0.31460725543703594
  (2, 36)	0.5846520797126125
  (2, 2373)	0.07732061406710827
  (2, 2729)	0.3253695476501475
  (2, 6084)	0.07749568479206446
  (2, 9301)	0.2554667597286382
  (2, 9302)	0.3690506889564557
  :	:
  (2155, 8841)	0.3788742673945968
  (2155, 8894)	0.2975278411757682
  (2155, 10887)	0.28517133262245253
  (2155, 12402)	0.27669248040492894
  (2155, 12554)	0.3788742673945968
  (2155, 13791)	0.282169972

In [381]:
# Model
model = LogisticRegression()

In [382]:
# Fitting the model
model.fit(X_train, Y_train)

LogisticRegression()

In [383]:
# Evaluating the model
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [384]:
# The accuracy
print("Accuracy score of the training data: ", training_data_accuracy)

Accuracy score of the training data:  0.8809055118110236


In [385]:
# Predicting Y_test
Y_test = model.predict(X_test)

In [386]:
# Printing Y_test in an array form
print(Y_test)

[1 0 1 ... 0 0 0]


In [387]:
# Getting the shape of Y_test
Y_test.shape

(2158,)

In [395]:
# Getting the value counts of Y_test
unique_values, counts = np.unique(Y_test, return_counts=True)

In [396]:
# Printing the value counts
print(dict(zip(unique_values, counts)))

{0: 1329, 1: 829}


In [388]:
# Adding these value to dataset_test in target column
dataset_test["target"] = Y_test

In [389]:
print(dataset_test.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2158 entries, 15 to 3250
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        2158 non-null   int64 
 1   keyword   2158 non-null   object
 2   location  2158 non-null   object
 3   text      2158 non-null   object
 4   content   2158 non-null   object
 5   target    2158 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 118.0+ KB
None


In [390]:
dataset_test.head()

,id,keyword,location,text,content,target
15,46,ablaze,London,Birmingham Wholesale Market is ablaze BBC News...,ablaz london birmingham wholesal market ablaz ...,1
16,47,ablaze,Niall's place | SAF 12 SQUAD |,@sunkxssedharry will you wear shorts for race ...,ablaz niall place saf squad sunkxssedharri wea...,0
17,51,ablaze,NIGERIA,#PreviouslyOnDoyinTv: Toke MakinwaÛªs marriag...,ablaz nigeria previouslyondoyintv toke makinwa...,1
18,58,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...,ablaz live webcam check http co roi nsmejj htt...,0
19,60,ablaze,"Los Angeles, Califnordia",PSA: IÛªm splitting my personalities.\n\n?? t...,ablaz lo angel califnordia psa split person te...,0


In [392]:
# Dropping content column
dataset_test = dataset_test.drop("content", axis=1)

In [393]:
dataset_test.head()

,id,keyword,location,text,target
15,46,ablaze,London,Birmingham Wholesale Market is ablaze BBC News...,1
16,47,ablaze,Niall's place | SAF 12 SQUAD |,@sunkxssedharry will you wear shorts for race ...,0
17,51,ablaze,NIGERIA,#PreviouslyOnDoyinTv: Toke MakinwaÛªs marriag...,1
18,58,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...,0
19,60,ablaze,"Los Angeles, Califnordia",PSA: IÛªm splitting my personalities.\n\n?? t...,0


In [394]:
# Exporting the new dataset as a csv file named result
dataset_test.to_csv("result.csv", index=False)